## CODIGO DE EJECUCION

*NOTA: Para poder usar este código de ejecución hay que lanzarlo desde exactamente el mismo entorno en el que fue creado.*

*Se puede instalar ese entorno en la nueva máquina usando el environment.yml que creamos en el set up del proyecto*

*Copiar el environment.yml que tenemos en la carpeta Documentos, al directorio y en anaconda prompt ejecutar:*

conda env create --file environment.yml --name proyecto1

* environment.yml --> Nombre del proyecto: 003_riesgos.yml
* name proyecto1 --> Nombre del proyecto: 003_riesgos

In [4]:
#1.LIBRERIAS
import os
import numpy as np
import pandas as pd
import pickle  

#Automcompletar rápido
%config IPCompleter.greedy=True

#Desactivar la notación científica
pd.options.display.float_format = '{:.2f}'.format

#Desactivar los warnings
import warnings
warnings.filterwarnings("ignore")

#Mostrar el máximo de filas posibles de una tabla
pd.set_option('display.max_rows', 100) #Número de filas que deben verse. None = Máx

#Mostrar mas caracteres de las columnas. Se usa cuando se corta el texto
pd.set_option('display.max_colwidth', None) #Número de caractres que deben verse. None = Máx

#Representación visual de un pipeline
from sklearn import set_config
set_config(display = 'diagram') #diagram/text 

#Transformación de variables
from janitor import clean_names
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

#Modelos ML: CLASIFICACIÓN
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#Métrica de error: Habrá que incluir la que necesitemos
from sklearn.metrics import roc_auc_score

#Modelos ML: REGRESIÓN
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
#Métrica de error: Habrá que incluir la que necesitemos
from sklearn.metrics import mean_absolute_error

#2.CARGA DATOS
ruta_proyecto = 'C:/Users/Oscar/OneDrive - FM4/Escritorio/Python Data Mastery/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/003_RIESGOS'
nombre_fichero_datos = 'validacion.csv'
ruta_completa = ruta_proyecto + '/02_Datos/02_Validacion/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,index_col='id_cliente')

#3.VARIABLES Y REGISTROS FINALES
variables_finales = ['antigüedad_empleo',
                     'ingresos_verificados',
                     'rating',
                     'vivienda',
                     'finalidad',
                     'num_cuotas',
                     'ingresos',
                     'dti',
                     'num_hipotecas',
                     'num_lineas_credito',
                     'porc_tarjetas_75p',
                     'porc_uso_revolving',
                     'num_cancelaciones_12meses',
                     'num_derogatorios',
                     'num_meses_desde_ult_retraso',
                     'principal',
                     'tipo_interes',
                     'imp_cuota',
                     'imp_amortizado',
                     'imp_recuperado',
                     'estado'
                    ]
a_eliminar = df.loc[df.ingresos > 300000].index.values
df = df[~df.index.isin(a_eliminar)]
df = df[variables_finales]

# En todo el resto del código tenemos que estar atentos porque no tenemos la target ni las variables de amortización, 
#recuperado y estado
#4.FUNCIONES DE SOPORTE.
def calidad_datos(temp):
    # Limpiar nombres de columnas
    temp = clean_names(temp)
    
    # Rellenar nulos de columna 'antiguedad_empleo'
    temp['antiguedad_empleo'] = temp['antiguedad_empleo'].fillna('desconocido')
    
    # Eliminar dato extraño
    #temp.drop(index=16130949)
   
    # Subfunción para categorizar 'porc_tarjetas_75p'
    def categorizar_tarjetas(porc):
        if porc <= 10:
            return '<10%'
        elif porc <= 75:
            return '11-75%'
        else:
            return '>75%'
    
    # Crear nueva columna categorizada
    if 'porc_tarjetas_75p' in temp.columns:
        temp['cat_tarjetas_75p'] = temp['porc_tarjetas_75p'].apply(categorizar_tarjetas)
    
    # Sustituimos por 0 los créditos que tienen los estados de impago y por 999 el resto que no han registrado nunca impagos
    temp['num_meses_desde_ult_retraso'] = np.where((temp.num_meses_desde_ult_retraso.isna())
                                               &(temp.estado.isin(['Charged Off',
                                                                   'Does not meet the credit policy. Status:Charged Off',
                                                                   'Default'])),
                                               temp['num_meses_desde_ult_retraso'].fillna(0),
                                               temp['num_meses_desde_ult_retraso'].fillna(999))
    
    # Rellenar nulos en el resto de columnas numéricas con 0
    for column in temp.select_dtypes('number').columns:
        temp[column] = temp[column].fillna(0)
    
    #Imputar por desviación típica
    def atipicos_desv_tip(variable, num_desv_tip = 4):
        #sacamos los nulos por ahora
        variable = variable.dropna()
        #calculamos los límites
        media = np.mean(variable)
        sd = np.std(variable)
        umbral = sd * num_desv_tip
        lim_inf = media - umbral
        lim_sup = media + umbral
        #encontramos los índices de los que están fuera de los límites
        indices = [indice for indice,valor in variable.items() if valor < lim_inf or valor > lim_sup]
        return(indices)
    var_atipicos_desv_tip = ['ingresos','num_hipotecas','principal','imp_cuota','imp_recuperado']
    
    #Winsorización manual
    temp['porc_uso_revolving'] = temp['porc_uso_revolving'].clip(0, 100)
    temp['dti'] = temp['dti'].clip(0, 100)    
    return temp

def creacion_variables(df):
    temp = df.copy()
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    return temp

#4.CALIDAD Y CREACION DE VARIABLES
x_pd = creacion_variables(calidad_datos(df))
x_ead = creacion_variables(calidad_datos(df))
x_lgd = creacion_variables(calidad_datos(df))


#5.CARGA PIPES DE EJECUCION
ruta_pipe_ejecucion_pd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_pd.pickle'
ruta_pipe_ejecucion_ead = ruta_proyecto + '/04_Modelos/pipe_ejecucion_ead.pickle'
ruta_pipe_ejecucion_lgd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_lgd.pickle'

with open(ruta_pipe_ejecucion_pd, mode='rb') as file:
   pipe_ejecucion_pd = pickle.load(file)

with open(ruta_pipe_ejecucion_ead, mode='rb') as file:
   pipe_ejecucion_ead = pickle.load(file)

with open(ruta_pipe_ejecucion_lgd, mode='rb') as file:
   pipe_ejecucion_lgd = pickle.load(file)


#6.EJECUCION
scoring_pd = pipe_ejecucion_pd.predict_proba(x_pd)[:, 1]
ead = pipe_ejecucion_ead.predict(x_ead)
lgd = pipe_ejecucion_lgd.predict(x_lgd)


#7.RESULTADO
#Fórmula sobre riesgo: Expected Loss = Probability of Default * Principal * Exposure at Default * Loss Given Default
#Fórmula sobre riesgo: EL = PD * P * EAD * LGD

#De toda la fórmula hemos hecho modelos para 'pd','ead' y 'lgd' y el principal sería la cantidad de dinero solicitada, por
#lo que ya tenemos todos los datos para saber Expected Loss

principal = x_pd.principal
EL = pd.DataFrame({'principal':principal,
                   'pd':scoring_pd,
                   'ead':ead,
                   'lgd':lgd                   
                   })
EL['perdida_esperada'] = round(EL.pd * EL.principal * EL.ead * EL.lgd,2)

In [6]:
EL.sort_values('perdida_esperada', ascending=False)

,principal,pd,ead,lgd,perdida_esperada
id_cliente,,,,,
131474877,40000.00,0.60,0.98,1.08,25177.06
119320772,35000.00,0.83,0.91,0.96,25063.15
123588058,32000.00,0.61,1.01,1.21,23876.04
111668452,35000.00,0.83,0.89,0.91,23532.10
77360244,37575.00,0.71,0.92,0.95,23472.70
...,...,...,...,...,...
498686,15250.00,0.20,-0.03,0.13,-10.51
59461499,28000.00,0.14,-0.06,0.06,-13.36
131994106,8400.00,0.46,-0.12,0.03,-16.55
